# Effect Size

## What is effect size
Effect size is a measure of strength between two variables

## P-Values v.s. Effect Size:
| P-Values    | Effect size   |
|-------------|---------------|
| Does a relationship exist?  | How strong is the relationship? |
| Comes from hypothesis test  | Separate from a hypothesis test |

## Minimum Detectable Effect (MDE)

The minimum difference between groups that an experiment can detect with an acceptable probability (power), given a sample size and a significance level.

The MDE answers the question: “How large does the effect need to be for me to detect it?”
In mean comparison tests, the MDE is expressed as **Cohen’s d**

#### Example
Computing the MDE
- sample size (nobs1) = 60
- power = 80%
- alpha = 5%
- alternative = two-sided (It indicates the presence of a difference, regardless of its direction (positive or negative))

In [15]:
from statsmodels.stats.power import TTestIndPower

analysis = TTestIndPower()

mde = analysis.solve_power(
    nobs1=60,
    power=0.8,
    alpha=0.05,
    alternative='two-sided'
)

print(f'Minimun detectable effect: {mde}')

Minimun detectable effect: 0.5157064989496001


## MDE in practical units (not just Cohen’s d)
In many cases, the MDE needs to be expressed in business-relevant units such as revenue, percentages, or conversion rates.

The formula is defined as **delta minimum = MDE * Standard Deviation**

Example:

In [16]:
mde = 0.42
std = 7.3
delta_min = mde * std

print(f'With this experimental design, only differences of {delta_min} units or greater can be detected; smaller differences will not be detectable.')

With this experimental design, only differences of 3.066 units or greater can be detected; smaller differences will not be detectable.


## Cohen's d - Effect size for means

![cohensd_formula](images/cohensd_formula.png)

![cohensd_interpreting](images/cohensd_interpreting.png)

## Exercise: Effect size for means
Many venture capital-backed companies receive more than one round of funding. In general, the second round is bigger than the first. Just how much of an effect does the round number have on the average funding amount? You can use Cohen's d to quantify this.

In [17]:
import pandas as pd
import numpy as np

In [18]:
investments_data = pd.read_csv('../data/investments_VC.csv')
investments_data.head(3)

,market,funding_total_usd,status,country_code,funding_rounds,seed,venture,equity_crowdfunding,private_equity
0,Games,4000000,operating,USA,2,0,4000000,0,0
1,Software,7000000,NaN,USA,1,0,7000000,0,0
2,Advertising,4912393,closed,ARG,1,0,0,0,0


In [19]:
# Select all investments from rounds 1 and 2 separately
round1_df = investments_data.loc[investments_data['funding_rounds'] == 1]
round2_df = investments_data.loc[investments_data['funding_rounds'] == 2]

# Calculate the standard deviation of each round and the number of companies in each round
round1_sd = round1_df['funding_total_usd'].std()
round2_sd = round2_df['funding_total_usd'].std()
round1_n = round1_df.shape[0]
round2_n = round2_df.shape[0]

# Calculate the pooled standard deviation between the two rounds
pooled_sd = np.sqrt(((round1_n - 1) * round1_sd ** 2 + (round2_n - 1) *round2_sd ** 2) / (round1_n + round2_n - 2))

# Calculate Cohen's d
d = (round1_df['funding_total_usd'].mean() - round2_df['funding_total_usd'].mean()) / pooled_sd
print(f"Cohen's d: {d}")

Cohen's d: -0.07719192881235906


#### Conclusion:
Cohen's d about 0.08 it's a surprisingly low value! That tells us that moving to a second round of funding does not in itself have a large effect on the amount of money raised. This is likely due to how large the standard deviations are, which means that the means are unreliable estimates.

**Note**: The sign of the result depends on the order in which the groups are defined in the subtraction. A negative value does not indicate a smaller effect size; rather, it indicates direction. In our example, the result can be interpreted as Group 1 having a lower mean than Group 2.

## Exercise: Effect size for correlations
### R-squared: Percent of variation in one variable explained by knowing the other.
The volatility of an asset is roughly defined by how much its price changes. In this exercise you'll measure volatility on a per-day basis, defined as the (high price - low price) / closing price.

What factors explain the volatility of Bitcoin? Is the volatility of the S&P500 closely related to this? Does volatility increase or decrease as prices rise? In other words, what is the effect size of the correlation between these different factors? You'll compute both of these effect size in this exercise.

In [20]:
from scipy.stats import pearsonr

btc_sp_data = pd.read_csv('../data/btc_sp.csv')

# Compute the volatility of Bitcoin
btc_sp_data['Volatility_BTC'] = (btc_sp_data['High_BTC']-btc_sp_data['Low_BTC'])/btc_sp_data['Close_BTC']

# Compute the volatility of the S&P500
btc_sp_data['Volatility_SP500'] = (btc_sp_data['High_SP500']-btc_sp_data['Low_SP500'])/btc_sp_data['Close_SP500']

# Compute and print R^2 between the volatility of BTC and SP500
r_volatility, p_value_volatility = pearsonr(btc_sp_data['Volatility_BTC'], btc_sp_data['Volatility_SP500'])
print('R^2 between volatility of the assets:', r_volatility**2)

# Compute and print R^2 between the volatility of BTC and the closing price of BTC
r_closing, p_value_closing = pearsonr(btc_sp_data['Volatility_BTC'], btc_sp_data['Close_BTC'])
print('R^2 between closing price and volatility of BTC:', r_closing**2)


R^2 between volatility of the assets: 0.03152987723111431
R^2 between closing price and volatility of BTC: 0.012520659135177218


### Conclusion:
The volatility in the S&P 500 has the greater effect on the volatility of Bitcoin.

## Exercise: Effect size for categorical variables

![cramersv_formula](images/cramersv_formula.png)

![interpreting_cramersv](images/interpreting_cramersv.png)

You saw in the City of Austin employee data that job titles have an unequal distribution of genders. But does the same thing hold for ethnicities? And to what extent does ethnicity relate to the job title chosen? In this exercise you'll dig in and answer that question.

In [21]:
austin_data = pd.read_csv('../data/austin_salaries_asset.csv')
austin_data.head()

,Title,Gender,Ethnicity,Annual Salary,Years of Employment
0,Administrative Specialist,F,White,51542.4,6
1,Administrative Specialist,M,Black or African American,48235.2,11
2,Administrative Specialist,F,Hispanic or Latino,51542.4,14
3,Administrative Specialist,F,Hispanic or Latino,48235.2,3
4,"MuniProg, Paraprofessional",F,White,50668.8,2


In [22]:
employees_df = pd.crosstab(
    austin_data['Title'],
    austin_data['Ethnicity']
)
employees_df

Ethnicity,Asian,Black or African American,Hispanic or Latino,White
Title,,,,
Administrative Specialist,5,34,99,78
Fire Specialist,8,9,36,149
Firefighter,5,37,127,361
"MuniProg, Paraprofessional",37,142,227,493
Police Corporal/Detective,5,31,77,263
Police Officer,34,86,263,788


While a chi-squared test can test for association between two categorical variables, it doesn't directly answer the question of the degree of association. By computing an effect size like Cramer's V, you can directly measure the effect that one variable has on the other.

In [23]:
from scipy.stats import chi2_contingency

# Compute the chi-squared statistic
chi2, p, d, expected = chi2_contingency(employees_df)

# Compute the DOF using the number of rows and columns
dof = min(employees_df.shape[0] - 1, employees_df.shape[1] - 1)

# Compute the total number of people
n = np.sum(employees_df.values)

# Compute Cramer's V
v = np.sqrt((chi2 / n) / dof)

print("Cramer's V:", v, "\nDegrees of freedom:", dof)


Cramer's V: 0.12902342923732754 
Degrees of freedom: 3


### Conclusion:
Ethnicity has a moderate effect on the job title a person holds.

Unlike R-squared, Cramer's V doesn't have an easy interpretation. But by knowing the degrees of freedom you can get a general feel for how large or small an effect one categorical variable has on another. Here you see that job title and ethnicity are certainly related, but not directly linked. This might lead you to search for what other factors are influencing the job a person gets.

## Cohen's h for difference in proportions

![cohens_h_formula](images/cohens_h.png)

#### Example (A/B test):

Suppose the following convertion rates:
- Email A does convert 12%
- Email B does convert 18%


In [24]:
from statsmodels.stats.proportion import proportion_effectsize

h = proportion_effectsize(0.12, 0.18)
print(h)

-0.1688148493889977


#### Conclusion:
The test yields a small effect size